Setting Up the Directory

In [ ]:
import os
# Set the directory you wish put OpenFold and Attention Viz Demo into, please change this to your desired root directory
os.environ['ROOT_DIR'] = '~/scratch'
# Set the directory that contains OpenFold data
os.environ['DATA_DIR'] = '/storage/ice1/shared/d-pace_community/alphafold/alphafold_2.3.2_data'
# Set the directory you wish put Conda
os.environ['CONDA_INSTALL_DIR'] = '~/scratch'

Fix paths to be expanded from user to exact paths

In [ ]:
os.environ['ROOT_DIR'] = os.path.expanduser(os.environ['ROOT_DIR'])
os.environ['DATA_DIR'] = os.path.expanduser(os.environ['DATA_DIR'])
os.environ['CONDA_INSTALL_DIR'] = os.path.expanduser(os.environ['CONDA_INSTALL_DIR'])

Clone the Attention Visualization Demo, and OpenFold repository.

In [ ]:
%%bash
# Clone the Attention Visualization Demo repository
git clone https://github.com/vizfold/attention-viz-demo.git $ROOT_DIR/attention-viz-demo
# Clone the OpenFold repository
git clone https://github.com/aqlaboratory/openfold.git $ROOT_DIR/openfold
# Fix environment permissions
sed -i '/^dependencies:/a \  - mamba' $ROOT_DIR/openfold/environment.yml 

Create and activate the OpenFold conda environment

In [ ]:
%%bash
# Change directory to OpenFold
cd $ROOT_DIR/openfold
# Activate Conda
source "$(conda info --base)/etc/profile.d/conda.sh"
export CONDA_ENVS_PATH=$CONDA_INSTALL_DIR/.conda/envs
export CONDA_PKGS_DIRS=$CONDA_INSTALL_DIR/.conda/pkgs
conda init
# Create the OpenFold conda environment
conda create -n openfold_env -c conda-forge mamba --no-default-packages -y
# Activate the OpenFold conda environment
conda activate openfold_env
# Update the OpenFold conda environment with environment.yml
mamba env update -n openfold_env -f environment.yml -y
# Add pip dependencies not installed by Mamba
pip install deepspeed==0.14.5 dm-tree==0.1.6 git+https://github.com/NVIDIA/dllogger.git flash-attn
echo "openfold_env created and activated"

Set up compiler and library paths

In [ ]:
%%bash
# Activate Conda
source "$(conda info --base)/etc/profile.d/conda.sh"
export CONDA_ENVS_PATH=$CONDA_INSTALL_DIR/.conda/envs
export CONDA_PKGS_DIRS=$CONDA_INSTALL_DIR/.conda/pkgs
conda init
conda activate openfold_env
# Change directory to OpenFold
cd $ROOT_DIR/openfold
# Set up compiler and library paths
mkdir -p $CONDA_PREFIX/x86_64-conda-linux-gnu/lib
ln -s $CONDA_PREFIX/libexec/gcc/x86_64-conda-linux-gnu/12.4.0/cc1plus $CONDA_PREFIX/bin/
ln -s $CONDA_PREFIX/lib/gcc/x86_64-conda-linux-gnu/12.4.0/crtbeginS.o $CONDA_PREFIX/x86_64-conda-linux-gnu/lib/crtbeginS.o
ln -s $CONDA_PREFIX/lib/gcc/x86_64-conda-linux-gnu/12.4.0/crtendS.o $CONDA_PREFIX/x86_64-conda-linux-gnu/lib/crtendS.o
ln -s $CONDA_PREFIX/x86_64-conda-linux-gnu/sysroot/usr/lib64/crti.o $CONDA_PREFIX/x86_64-conda-linux-gnu/lib/crti.o
ln -s $CONDA_PREFIX/x86_64-conda-linux-gnu/sysroot/usr/lib64/crtn.o $CONDA_PREFIX/x86_64-conda-linux-gnu/lib/crtn.o
# Install gcc and libgcc-ng
mamba install -y gcc_linux-64 libgcc-ng
# Set up environment variables
export GCC_LTO_PLUGIN="$CONDA_PREFIX/libexec/gcc/x86_64-conda-linux-gnu/12.4.0/liblto_plugin.so"
export CFLAGS="-O2 -fno-lto --sysroot=$CONDA_PREFIX/x86_64-conda-linux-gnu/sysroot"
export CXXFLAGS="$CXXFLAGS -fno-use-linker-plugin -O2 -fno-lto --sysroot=$CONDA_PREFIX/x86_64-conda-linux-gnu/sysroot"
export CFLAGS="$CFLAGS -fno-use-linker-plugin -O2 -fno-lto --sysroot=$CONDA_PREFIX/x86_64-conda-linux-gnu/sysroot"
export LDFLAGS="$LDFLAGS -fno-use-linker-plugin -O2 -fno-lto --sysroot=$CONDA_PREFIX/x86_64-conda-linux-gnu/sysroot"
export LDFLAGS="$LDFLAGS -L$CONDA_PREFIX/lib/gcc/x86_64-conda-linux-gnu/12.4.0 -L$CONDA_PREFIX/x86_64-conda-linux-gnu/sysroot/usr/lib64"
export CPATH="$CONDA_PREFIX/include:$CPATH"
export LIBRARY_PATH="$CONDA_PREFIX/lib:$LIBRARY_PATH"
export LD_LIBRARY_PATH="$CONDA_PREFIX/lib:$LD_LIBRARY_PATH"
# Install OpenFold
pip install -e .
# Install third-party dependencies
scripts/install_third_party_dependencies.sh

Set up Attention Visualization Demo

In [ ]:
%%bash
# Activate Conda
source "$(conda info --base)/etc/profile.d/conda.sh"
export CONDA_ENVS_PATH=$CONDA_INSTALL_DIR/.conda/envs
export CONDA_PKGS_DIRS=$CONDA_INSTALL_DIR/.conda/pkgs
conda init
conda activate openfold_env
# Change directory to OpenFold
cd $ROOT_DIR/openfold
# Install additional required packages
mamba install -y ipykernel
python -m ipykernel install --user --name=openfold_env
# Set up Attention Visualization Demo
cd $ROOT_DIR/attention-viz-demo
ln -s $ROOT_DIR/openfold/openfold/data ./openfold/
# Create necessary directories and symlinks
mkdir openfold/resources
ln -s $DATA_DIR/params ./openfold/resources/ 
wget -N --no-check-certificate -P openfold/resources https://git.scicore.unibas.ch/schwede/openstructure/-/raw/7102c63615b64735c4941278d92b554ec94415f8/modules/mol/alg/src/stereo_chemical_props.txt

Install additional visualization tools

In [ ]:
%%bash
# Activate Conda
source "$(conda info --base)/etc/profile.d/conda.sh"
export CONDA_ENVS_PATH=$CONDA_INSTALL_DIR/.conda/envs
export CONDA_PKGS_DIRS=$CONDA_INSTALL_DIR/.conda/pkgs
conda init
conda activate openfold_env
# Install matplotlib
mamba install -y conda-forge::matplotlib
# Set strict channel priority for consistent package resolution
conda config --set channel_priority strict 
# Install PyTorch with CUDA support
mamba install -y pytorch=2.5 pytorch-cuda=12.4 -c pytorch -c nvidia
# Install PyMOL for molecular visualization
mamba install -y -c conda-forge -c pytorch -c nvidia pymol-open-source
# Reset channel priority
conda config --remove-key channel_priority